<a href="https://colab.research.google.com/github/gmelaku/Assignment1/blob/master/MP5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework-5: Data Cleaning
## Adapted from: https://github.com/XD-DENG/Spark-practice
## 1: Please upload "2015-12-12.csv" (provided at the beggining of the HW-5 instructions) inside a "sample_data" directory
## 2: Fill out right hand side below each # TODO comments in the code. 

If you do not have spark-hadoop setup properly on your system, use follwing lines of code to download and setup:


In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

# !tar -xvf /content/spark-2.4.5-bin-hadoop2.7.tgz
!tar -xvf  spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

import findspark
findspark.init()

# Create a blank SparkContext
from pyspark import SparkContext
sc = SparkContext()

# Upload the csv file into the notebook

In [2]:
from google.colab import files
uploaded = files.upload()

Saving 2015-12-12.csv to 2015-12-12.csv


## 1] Upload a csv file into a RDD and print how many entries are there

In [3]:
#TODO: Load the file
raw_content = sc.textFile("2015-12-12.csv")
print(type(raw_content))
# TODO print total entries in the RDD
raw_content.count()


<class 'pyspark.rdd.RDD'>


421970

### 2] Use take() and takeSample() on the loaded RDD content in order to check the difference between both calls

In [4]:
# TODO: use take() for getting 5 rows

raw_content.take(5)



['"date","time","size","r_version","r_arch","r_os","package","version","country","ip_id"',
 '"2015-12-12","13:42:10",257886,"3.2.2","i386","mingw32","HistData","0.7-6","CZ",1',
 '"2015-12-12","13:24:37",1236751,"3.2.2","x86_64","mingw32","RJSONIO","1.3-0","DE",2',
 '"2015-12-12","13:42:35",2077876,"3.2.2","i386","mingw32","UsingR","2.0-5","CZ",1',
 '"2015-12-12","13:42:01",266724,"3.2.2","i386","mingw32","gridExtra","2.0.0","CZ",1']

In [5]:
# TODO: use takeSample() for sampling 5 entries
raw_content.takeSample(True,5, 3)

['"2015-12-12","05:59:49",511365,"3.2.3","i386","mingw32","ks","1.10.0","US",10364',
 '"2015-12-12","05:10:24",287843,"3.2.2","x86_64","mingw32","xtable","1.8-0","US",5665',
 '"2015-12-12","13:06:32",494138,"3.2.3","x86_64","linux-gnu","rjson","0.2.15","KR",655',
 '"2015-12-12","23:29:36",1350975,"3.2.3","x86_64","mingw32","randtoolbox","1.17","US",6509',
 '"2015-12-12","13:13:10",45187,"3.2.3","x86_64","linux-gnu","spatial","7.3-11","GB",548']

## 3] Split each row in the raw content RDD by a comma or "," and remove double quotes around the string value from the columns 

In [6]:

# Split each row in the raw content RDD by a comma
content = raw_content.map(lambda x: x.split(','))
content.take(5)


[['"date"',
  '"time"',
  '"size"',
  '"r_version"',
  '"r_arch"',
  '"r_os"',
  '"package"',
  '"version"',
  '"country"',
  '"ip_id"'],
 ['"2015-12-12"',
  '"13:42:10"',
  '257886',
  '"3.2.2"',
  '"i386"',
  '"mingw32"',
  '"HistData"',
  '"0.7-6"',
  '"CZ"',
  '1'],
 ['"2015-12-12"',
  '"13:24:37"',
  '1236751',
  '"3.2.2"',
  '"x86_64"',
  '"mingw32"',
  '"RJSONIO"',
  '"1.3-0"',
  '"DE"',
  '2'],
 ['"2015-12-12"',
  '"13:42:35"',
  '2077876',
  '"3.2.2"',
  '"i386"',
  '"mingw32"',
  '"UsingR"',
  '"2.0-5"',
  '"CZ"',
  '1'],
 ['"2015-12-12"',
  '"13:42:01"',
  '266724',
  '"3.2.2"',
  '"i386"',
  '"mingw32"',
  '"gridExtra"',
  '"2.0.0"',
  '"CZ"',
  '1']]

In [7]:

# Remove double quotes around string value from the column
def clean(x):
  return([xx.replace('"', '') for xx in x])

content = content.map(clean)

content.take(5)

[['date',
  'time',
  'size',
  'r_version',
  'r_arch',
  'r_os',
  'package',
  'version',
  'country',
  'ip_id'],
 ['2015-12-12',
  '13:42:10',
  '257886',
  '3.2.2',
  'i386',
  'mingw32',
  'HistData',
  '0.7-6',
  'CZ',
  '1'],
 ['2015-12-12',
  '13:24:37',
  '1236751',
  '3.2.2',
  'x86_64',
  'mingw32',
  'RJSONIO',
  '1.3-0',
  'DE',
  '2'],
 ['2015-12-12',
  '13:42:35',
  '2077876',
  '3.2.2',
  'i386',
  'mingw32',
  'UsingR',
  '2.0-5',
  'CZ',
  '1'],
 ['2015-12-12',
  '13:42:01',
  '266724',
  '3.2.2',
  'i386',
  'mingw32',
  'gridExtra',
  '2.0.0',
  'CZ',
  '1']]

## 4] Reduce and Counting: work on the RDD generated in Question-3
### 1. Get the RDD for counting downloading records for each package (7th column): use reduceByKey(). Get how many distinct packages are there? Also print the count for 'dbmss' package.
### 2. Get the RDD for counting downloading records for each package (7th column): use countByKey(). Print the count for 'dbmss' package.
### 3. As countByKey() works only on RDDs of type (K, V), check the effect of the value of V by using different values for V and print the count for 'dbmss' package after each case to check the effect of a value for V.

# 4.1

In [8]:
# Get the number of packages of 7th column using reduceByKey()
package_count = content.map(lambda x: (x[6], 1)).reduceByKey(lambda a,b: a+b)
#print the type
print (type(package_count))
#the total number of distinct packages
package_count.count()
#Take the first five values for package_count
package_count.take(5)


<class 'pyspark.rdd.PipelinedRDD'>


[('HistData', 159),
 ('UsingR', 151),
 ('lme4', 1560),
 ('testthat', 1178),
 ('maps', 1586)]

# 4.2

In [9]:
#Use countByKey to perform the above function
package_count_2 = content.map(lambda x: (x[6], 1)).countByKey()
#print the type for package count using countByKey
print(type(package_count_2))
#Count the number of dbmss
package_count_2['dbmss']

<class 'collections.defaultdict'>


54

# 4.3

Using v =2

In [10]:
#Use countByKey to perform the above function
package_count_3 = content.map(lambda x: (x[6], 2)).countByKey()
#print the type for package count using countByKey
print(type(package_count_3))
#Count the number of dbmss
package_count_3['dbmss']

<class 'collections.defaultdict'>


54

Using v =5

In [11]:
#Use countByKey to perform the above function
package_count_4 = content.map(lambda x: (x[6], 5)).countByKey()
#print the type for package count using countByKey
print(type(package_count_4))
#Count the number of dbmss
package_count_4['dbmss']

<class 'collections.defaultdict'>


54

## 5] Sorting: Work on the RDD generated in Question 4-1.
### 1. On the package count RDD, use sortByKey() to sort in descending order and print first-5 or 10 entries
### 2. On the package count RDD, use sortByKey() to sort in ascending order and print first-5 or 10 entries
### 3. On the package count RDD, use sortBy() to sort in descending order and print first-5 or 10 entries
### 4. On the package count RDD, use sortBy() to sort in ascending order and print first-5 or 10 entries

# 5.1

In [12]:

# Sort in descending order and print the first 10 entries
package_count.map(lambda x: (x[1], x[0])).sortByKey(0).take(10)


[(4783, 'Rcpp'),
 (3913, 'ggplot2'),
 (3748, 'stringi'),
 (3449, 'stringr'),
 (3436, 'plyr'),
 (3265, 'magrittr'),
 (3223, 'digest'),
 (3205, 'reshape2'),
 (3046, 'RColorBrewer'),
 (3007, 'scales')]

# 5.2

In [13]:

# Sort in ascending order and print the first 10 entries
package_count.map(lambda x: (x[1], x[0])).sortByKey(1).take(10)

[(1, 'multic'),
 (1, 'RBerkeley'),
 (1, 'vimcom'),
 (1, 'waldwolf'),
 (1, 'bstats'),
 (1, 'parspatstat'),
 (1, 'WaveCGH'),
 (1, 'mixnet'),
 (1, 'postgwas'),
 (1, 'rolasized')]

# 5.3

In [14]:

# sortBy in descending order
package_count.sortBy(lambda x:x[1], ascending = False).take(10)

[('Rcpp', 4783),
 ('ggplot2', 3913),
 ('stringi', 3748),
 ('stringr', 3449),
 ('plyr', 3436),
 ('magrittr', 3265),
 ('digest', 3223),
 ('reshape2', 3205),
 ('RColorBrewer', 3046),
 ('scales', 3007)]

# 5.4

In [15]:
# sortBy in ascending order
package_count.sortBy(lambda x:x[1]).take(10)

[('multic', 1),
 ('RBerkeley', 1),
 ('vimcom', 1),
 ('waldwolf', 1),
 ('bstats', 1),
 ('parspatstat', 1),
 ('WaveCGH', 1),
 ('mixnet', 1),
 ('postgwas', 1),
 ('rolasized', 1)]

## 6] Filter: Work on the RDD generated in Question 3; Get the date, time and size for the rows with package is 'Rtts' and country is 'US'

In [16]:
# Work on Question 6
content.filter(lambda x: x[6] == 'Rtts' and x[8] == 'US').count()
content.filter(lambda x: x[6] == 'Rtts' and x[8] == 'US').take(2)


[['2015-12-12',
  '04:52:36',
  '23820',
  '3.2.3',
  'i386',
  'mingw32',
  'Rtts',
  '0.3.3',
  'US',
  '1652'],
 ['2015-12-12',
  '20:31:45',
  '23820',
  '3.2.3',
  'x86_64',
  'linux-gnu',
  'Rtts',
  '0.3.3',
  'US',
  '4438']]

## 7] Set operations: Work on the raw content RDD from Question 1
### 1. Union: union raw content RDD (raw_content) with itself and get the total count after the union operation
### 2. Intersect: get the distinct value set of raw content RDD using intersect operation
### 3. Use the distinct() function to get the count of distinct rows

# 7.1

In [17]:
# Union
raw_content.union(raw_content)
#Get count after the union
raw_content.union(raw_content).count()


843940

# 7.2

In [18]:
# Intersection
raw_content.intersection(raw_content)
# Get the distinct counts after intersection 
raw_content.intersection(raw_content).count()

421553

# 7.3

In [19]:
 # Distinct values
raw_content.distinct().count()

421553

## 8] Join: Work on the RDD generated in Question 3
### 1. Generate a new RDD in which the values from the 'country' column is a KEY
### 2. For a given mapping of the abbreviates of four countries and their full name perform join and left outer join using join() and leftOuterJoin() functions. Also sample 8 entries from each.

# Set column containing country as KEY

In [0]:

content2=content.map(lambda x:(x[8], x))

Give a mapping table of the four countries and their full names

In [0]:
mapping=[('DE', 'Germany'), ('US', 'United States'), ('CN', 'China'), ('IN',"India")]
mapping=sc.parallelize(mapping)

# Work on Question 8


# 8.1

In [22]:
# join and sample 8 entries
content2.join(mapping).takeSample(False, 8)

[('IN',
  (['2015-12-12',
    '05:50:11',
    '27211',
    '3.2.3',
    'x86_64',
    'linux-gnu',
    'RColorBrewer',
    '1.1-2',
    'IN',
    '10566'],
   'India')),
 ('US',
  (['2015-12-12',
    '20:22:53',
    '1328404',
    '3.2.3',
    'x86_64',
    'linux-gnu',
    'PKI',
    '0.1-3',
    'US',
    '7899'],
   'United States')),
 ('US',
  (['2015-12-12',
    '18:08:23',
    '46469',
    '3.2.2',
    'x86_64',
    'mingw32',
    'fpp',
    '0.5',
    'US',
    '9843'],
   'United States')),
 ('CN',
  (['2015-12-12',
    '11:15:23',
    '14616',
    'NA',
    'NA',
    'NA',
    'xlsxjars',
    '0.6.1',
    'CN',
    '483'],
   'China')),
 ('US',
  (['2015-12-12',
    '04:51:06',
    '4300329',
    '3.2.3',
    'x86_64',
    'darwin13.4.0',
    'XML',
    '3.98-1.3',
    'US',
    '2039'],
   'United States')),
 ('US',
  (['2015-12-12',
    '22:50:48',
    '631878',
    '3.2.3',
    'x86_64',
    'linux-gnu',
    'multcomp',
    '1.4-1',
    'US',
    '7608'],
   'United States'

# 8.2

In [23]:
# Left outer join and take the first 8 sample
content2.leftOuterJoin(mapping).takeSample(False, 8)

[('GB',
  (['2015-12-12',
    '20:05:11',
    '3248123',
    'NA',
    'NA',
    'NA',
    'psych',
    '1.5.8',
    'GB',
    '13819'],
   None)),
 ('TN',
  (['2015-12-12',
    '12:36:50',
    '164040',
    '3.2.3',
    'x86_64',
    'mingw32',
    'Formula',
    '1.2-1',
    'TN',
    '3479'],
   None)),
 ('US',
  (['2015-12-12',
    '01:34:03',
    '711706',
    'NA',
    'NA',
    'NA',
    'rJava',
    '0.9-7',
    'US',
    '2940'],
   'United States')),
 ('CN',
  (['2015-12-12',
    '20:08:54',
    '510',
    'NA',
    'NA',
    'NA',
    'rrv',
    '0.0.2',
    'CN',
    '6339'],
   'China')),
 ('CN',
  (['2015-12-12',
    '07:07:02',
    '321913',
    '3.2.2',
    'x86_64',
    'mingw32',
    'kknn',
    '1.3.0',
    'CN',
    '363'],
   'China')),
 ('GB',
  (['2015-12-12',
    '23:59:35',
    '3195827',
    '3.2.3',
    'i386',
    'mingw32',
    'Rcpp',
    '0.12.2',
    'GB',
    '11544'],
   None)),
 ('CN',
  (['2015-12-12',
    '22:25:05',
    '511',
    'NA',
    'NA',
 